# 8.1 层次化索引

In [5]:
import pandas as pd
import numpy as np
data = pd.Series(np.random.randn(9),
                index=[['a','a','a','b','b','c','c','d','d'],
                      [1,2,3,1,3,1,2,2,3]])
data

a  1   -0.503891
   2   -0.796135
   3    0.032854
b  1    0.275525
   3    0.773185
c  1   -0.071189
   2    0.320788
d  2   -1.204794
   3    1.331400
dtype: float64

In [6]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [7]:
data['b']

1    0.275525
3    0.773185
dtype: float64

In [8]:
data['b':'c']

b  1    0.275525
   3    0.773185
c  1   -0.071189
   2    0.320788
dtype: float64

In [9]:
data.loc[['b','d']]

b  1    0.275525
   3    0.773185
d  2   -1.204794
   3    1.331400
dtype: float64

In [10]:
data.loc[:,2]

a   -0.796135
c    0.320788
d   -1.204794
dtype: float64

In [11]:
data.unstack()

,1,2,3
a,-0.503891,-0.796135,0.032854
b,0.275525,NaN,0.773185
c,-0.071189,0.320788,NaN
d,NaN,-1.204794,1.331400


In [12]:
data.unstack().stack()

a  1   -0.503891
   2   -0.796135
   3    0.032854
b  1    0.275525
   3    0.773185
c  1   -0.071189
   2    0.320788
d  2   -1.204794
   3    1.331400
dtype: float64

In [15]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index=[['a','a','b','b'],[1,2,3,4]]
                    ,columns=[['Ohio','Ohio','Colorado'],
                             ['Green','Red','Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 3     6   7        8
  4     9  10       11

In [16]:
frame.index.names = ['key1','key2']
frame.columns.names = ['state','color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    3        6   7        8
     4        9  10       11

In [17]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    3         6    7
     4         9   10

# 重排与分级排序

In [18]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
3    b        6   7        8
4    b        9  10       11

In [19]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    3        6   7        8
     4        9  10       11

In [20]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
3    b        6   7        8
4    b        9  10       11

# 根据级别汇总统计

In [21]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         0   1        2
2         3   4        5
3         6   7        8
4         9  10       11

In [22]:
frame.sum(level='color',axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    3        14    7
     4        20   10

# 使用DataFrame的列进行索引

In [23]:
frame = pd.DataFrame({'a':range(7),'b':range(7,0,-1)
                     ,'c':['one','one','one','two','two','two','two'],
                     'd':[0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [24]:
frame2 = frame.set_index(['c','d'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [25]:
frame.set_index(['c','d'],drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [26]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 8.2 合并数据集

# 数据库风格的DataFrame合并

In [27]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b']
                   ,'data1':range(7)})
df2 = pd.DataFrame({'key':['a','b','d'],'data2':range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [28]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [29]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [30]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [31]:
df3 = pd.DataFrame({'lkey':['b','b','a','c','a','c','a']
                   ,'data1':range(7)})
df4 = pd.DataFrame({'rkey':['a','b','d'],
                   'data2':range(3)})
pd.merge(df3,df4,left_on='lkey',right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,6,a,0


In [32]:
pd.merge(df1,df2,how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [33]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','b'],
                   'data1':range(6)})
df2 = pd.DataFrame({'key':['a','b','a','b','d'],
                   'data2':range(5)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [34]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [35]:
pd.merge(df1,df2,on='key',how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [36]:
pd.merge(df1,df2,how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [37]:
left = pd.DataFrame({'key1':['foo','foo','bar'],
                    'key2':['one','two','one'],
                    'lval':[1,2,3]})
right = pd.DataFrame({'key1':['foo','foo','bar','bar'],
                    'key2':['one','two','one','two'],
                    'lval':[4,5,6,7]})
pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,lval_x,lval_y
0,foo,one,1.0,4
1,foo,two,2.0,5
2,bar,one,3.0,6
3,bar,two,NaN,7


In [38]:
pd.merge(left,right,on='key1')

,key1,key2_x,lval_x,key2_y,lval_y
0,foo,one,1,one,4
1,foo,one,1,two,5
2,foo,two,2,one,4
3,foo,two,2,two,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [41]:
pd.merge(left,right,on='key1',suffixes=('_left','_right'))

,key1,key2_left,lval_left,key2_right,lval_right
0,foo,one,1,one,4
1,foo,one,1,two,5
2,foo,two,2,one,4
3,foo,two,2,two,5
4,bar,one,3,one,6
5,bar,one,3,two,7


# 索引上的合并

In [42]:
left1 = pd.DataFrame({'key':['a','b','a','a','b','c'],
                     'value':range(6)})
right1 = pd.DataFrame({'group_val':[3.5,7]},index=['a','b'])
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [45]:
right1

,group_val
a,3.5
b,7.0


In [46]:
pd.merge(left1,right1,left_on='key',right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [50]:
lefth = pd.DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],
                     'key2':[2000,2001,2002,2001,2002],
                     'data':np.arange(5)})
righth = pd.DataFrame(np.arange(12).reshape((6,2)),
                      index=[['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],
                            [2001,2000,2000,2000,2001,2002]],
                            columns=['event1','event2'])
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [51]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [52]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [53]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True,how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [54]:
left2 = pd.DataFrame([[1,2],[3,4],[5,6]],index=['a','c','e'],
                    columns=['Ohio','Nevada'])
right2 = pd.DataFrame([[7,8],[9,10],[11,12],[13,14]],
                     index=['b','c','d','e'],
                     columns=['Missouri','Alabama'])
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [55]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [56]:
pd.merge(left2,right2,how='outer',left_index=True,right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [57]:
left2.join(right2,how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [58]:
left1.join(right1,on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [59]:
another = pd.DataFrame([[7,8],[9,10],[11,12],[13,14]],index=['a','c','e','f'],
                      columns=['New York','Oregon'])
another

,New York,Oregon
a,7,8
c,9,10
e,11,12
f,13,14


In [60]:
left2.join([right2,another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,NaN,NaN,7,8
c,3,4,9.0,10.0,9,10
e,5,6,13.0,14.0,11,12


In [61]:
left2.join([right2,another],how='outer')

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:6848: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,13.0,14.0


# 轴向连接

In [62]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [63]:
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [64]:
s1 = pd.Series([0,1],index=['a','b'])
s2 = pd.Series([2,3,4],index=['c','d','e'])
s3 = pd.Series([5,6],index=['f','g'])
pd.concat([s1,s2,s3])
#concat默认在axis=0上工作

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [65]:
#如果传入axis=1，结果会变成DataFrame
pd.concat([s1,s2,s3],axis=1)

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [67]:
s4 = pd.concat([s1,s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [68]:
pd.concat([s1,s4],axis=1)

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [69]:
pd.concat([s1,s4],axis=1,join='inner')

,0,1
a,0,0
b,1,1


In [70]:
pd.concat([s1,s4],axis=1,join_axes=[['a','c','b','e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [71]:
result = pd.concat([s1,s1,s3],keys=['one','two','three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [72]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [73]:
pd.concat([s1,s2,s3],axis=1,keys=['one','two','three'])

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [76]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2),index=['a','b','c']
                   ,columns=['one','two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2),index=['a','c']
                        ,columns=['three','four'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [77]:
df2

,three,four
a,5,6
c,7,8


In [78]:
pd.concat([df1,df2],axis=1,keys=['level1','level2'])

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [79]:
pd.concat({'level1':df1,'level2':df2},axis=1)

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [81]:
pd.concat([df1,df2],axis=1,keys=['level1','level2'],
         names=['upper','lower'])

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [83]:
df1 = pd.DataFrame(np.random.randn(3,4),columns=['a','b','c','d'])
df2 = pd.DataFrame(np.random.randn(2,3),columns=['b','a','c'])
df1

,a,b,c,d
0,0.053563,0.431128,0.276605,1.555250
1,-1.750411,0.298010,-2.141098,-0.041550
2,1.716206,1.463535,1.365120,0.236001


In [84]:
df2

,b,a,c
0,0.629057,0.752150,0.796330
1,0.427972,-0.551257,-1.455598


In [86]:
pd.concat([df1,df2],ignore_index=True)

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,0.053563,0.431128,0.276605,1.555250
1,-1.750411,0.298010,-2.141098,-0.041550
2,1.716206,1.463535,1.365120,0.236001
3,0.752150,0.629057,0.796330,NaN
4,-0.551257,0.427972,-1.455598,NaN


# 合并重叠数据

In [89]:
a = pd.Series([np.nan,2.5,np.nan,3.5,4.5,np.nan],
             index=['f','e','d','c','b','a'])
b = pd.Series(np.arange(len(a),dtype=np.float64),
             index=['f','e','d','c','b','a'])
b[-1] = np.nan
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [90]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [91]:
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [92]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [93]:
df1 = pd.DataFrame({'a':[1,np.nan,5,np.nan],
                   'b':[np.nan,2,np.nan,6],
                   'c':range(2,18,4)})
df2 = pd.DataFrame({'a':[5,4,np.nan,3,7],
                   'b':[np.nan,3,4,6,8]})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [94]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [95]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# 8.3 重塑和轴向旋转

# 重塑层次化索引

In [112]:
#stack将数据的列旋转为行
#unstack将数据的行旋为列
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                   index=pd.Index(['Ohio','Colorado']
                                 ,columns=pd.Index(['one','two','three'])
                                 ,name='number'))
data

,0,1,2
number,,,
Ohio,0,1,2
Colorado,3,4,5


In [113]:
result = data.stack()
result

number     
Ohio      0    0
          1    1
          2    2
Colorado  0    3
          1    4
          2    5
dtype: int32

In [104]:
result.unstack()

number,one,two,three
Ohio,0,1,2
Colorado,3,4,5


In [106]:
result.unstack(0)

,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [115]:
s1 = pd.Series([0,1,2,3],index=['a','b','c','d'])
s2 = pd.Series([4,5,6],index=['c','d','e'])
data2 = pd.concat([s1,s2],keys=['one','two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [116]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [117]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [118]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

# 将“长格式”旋转为“宽格式”

In [2]:
import pandas as pd
data = pd.read_csv('C:\Mine\Github\Python\macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [6]:
periods = pd.PeriodIndex(year=data.year,quarter=data.quarter,name='date')
columns = pd.Index(['realgdp','infl','unemp'],name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D','end')
ldata = data.stack().reset_index().rename(columns={0:})
pivoted = ldata.pivot('date','item','value')
pivoted

SyntaxError: invalid syntax (<ipython-input-6-3846a4431689>, line 5)

In [8]:
import numpy as np
ldata['valude2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,2,valude2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-1.444925
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.070485
2,1959-03-31 23:59:59.999999999,unemp,5.800,1.176280
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.717854
4,1959-06-30 23:59:59.999999999,infl,2.340,-1.604885
5,1959-06-30 23:59:59.999999999,unemp,5.100,-1.379015
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.482019
7,1959-09-30 23:59:59.999999999,infl,2.740,0.789921
8,1959-09-30 23:59:59.999999999,unemp,5.300,1.080532
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,0.846215


In [9]:
pivoted = ldata.pivot('date','item')
pivoted[:5]

2                   valude2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.070485 -1.444925   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -1.604885 -0.717854   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.789921  0.482019   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.752156  0.846215   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  1.448078 -0.153374   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.176280  
1959-06-30 23:59:59.999999999 -1.379015  
1959-09-30 23:59:59.999999999  1.080532  
1959-12-31 23:59:59.999999999  1.082980  
1960-03-31 23:59:59.999999999  0.275685

In [22]:
pivoted[:5]

2                   valude2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.070485 -1.444925   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -1.604885 -0.717854   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.789921  0.482019   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.752156  0.846215   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  1.448078 -0.153374   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.176280  
1959-06-30 23:59:59.999999999 -1.379015  
1959-09-30 23:59:59.999999999  1.080532  
1959-12-31 23:59:59.999999999  1.082980  
1960-03-31 23:59:59.999999999  0.275685

In [17]:
#pivot其实就是用set_index创建层次化索引，再用unstack重塑
unstacked = ldata.set_index(['date','item']).unstack()
unstacked[:7]

2                   valude2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.070485 -1.444925   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -1.604885 -0.717854   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.789921  0.482019   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.752156  0.846215   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  1.448078 -0.153374   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -1.929970 -0.218009   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6  1.033654 -0.137874   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.176280  
1959-06-30 23:59:59.999999999 -1.379015  
1959-09-30 23:59:59.999999999  1.080532  
1959-12-31 23:59:59.999999999  1.082980  
1960-03-31 23:59:59.999999999  0.275685  
1960-06-30 23:59:59.999999999 -0.306999  
1960-09-30 23:59:59.999999999  0.217522

# 将“宽格式”旋转为“长格式”

In [120]:
df = pd.DataFrame({'key':['foo','bar','baz'],
                  'A':[1,2,3]
                  ,'B':[4,5,6],
                  'C':[7,8,9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [122]:
melted = pd.melt(df,['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [123]:
#使用pivot，可以重塑回原来的样子
reshaped = melted.pivot('key','variable','value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [124]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [126]:
pd.melt(df,id_vars=['key'],value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [127]:
pd.melt(df,value_vars=['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [130]:
pd.melt(df,value_vars=['key','A','B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
